In [22]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import rioxarray as rio
from pathlib import Path
import pandas as pd
import zarr

In [23]:
#done
#!wget https://zenodo.org/record/8055879/files/seasfire_v0.3.zip

In [24]:
#!unzip seasfire_v0.3.zip
#!ls

In [25]:
dataset=xr.open_dataset(Path.home() / 'Documents/Wildfire Project/SeasFireCube_v3.zarr', engine='zarr')

In [26]:
dataset

<xarray.Dataset>
Dimensions:                         (latitude: 720, longitude: 1440, time: 966)
Coordinates:
  * latitude                        (latitude) float64 89.88 89.62 ... -89.88
  * longitude                       (longitude) float64 -179.9 -179.6 ... 179.9
  * time                            (time) datetime64[ns] 2001-01-01 ... 2021...
Data variables: (12/59)
    area                            (latitude, longitude) float32 ...
    biomes                          (latitude, longitude) float32 ...
    cams_co2fire                    (time, latitude, longitude) float32 ...
    cams_frpfire                    (time, latitude, longitude) float32 ...
    drought_code_max                (time, latitude, longitude) float32 ...
    drought_code_mean               (time, latitude, longitude) float32 ...
    ...                              ...
    t2m_max                         (time, latitude, longitude) float32 ...
    t2m_mean                        (time, latitude, longitude) float32 ...
    t2m_min                         (time, latitude, longitude) float32 ...
    tp                              (time, latitude, longitude) float32 ...
    vpd                             (time, latitude, longitude) float32 ...
    ws10                            (time, latitude, longitude) float32 ...
Attributes:
    crs:          EPSG:4326
    description:  The SeasFire Cube is a scientific datacube for seasonal fir...
    title:        SeasFire Cube: A Global Dataset for Seasonal Fire Modeling ...

In [27]:

# Clip the xarray dataset by getting min and max lat and long and then passing it through the rio built in functions to clip
def get_clipped_xarray_dataset(ds, min_long, min_lat, max_long, max_lat):
    # CRS code 
    dsc = ds.rio.write_crs(4326) # Need to specify this as Ilektra mentioned 
    
    # Clip the dataset
    ds_clipped = dsc.rio.clip_box(minx= min_long, miny= min_lat, maxx= max_long,maxy= max_lat)
    
    return ds_clipped 

#California
ds_clipped = get_clipped_xarray_dataset(dataset, -124,32,-114,42)

In [28]:
ds_clipped.fcci_ba

<xarray.DataArray 'fcci_ba' (time: 966, latitude: 40, longitude: 40)>
[1545600 values with dtype=float32]
Coordinates:
  * latitude     (latitude) float64 41.88 41.62 41.38 ... 32.62 32.38 32.12
  * longitude    (longitude) float64 -123.9 -123.6 -123.4 ... -114.4 -114.1
    spatial_ref  int64 0
  * time         (time) datetime64[ns] 2001-01-01 2001-01-09 ... 2021-12-27
Attributes:
    creator_notes:    Masked ocean with lsm variable. Missing years filled wi...
    description:      The ESA Fire Disturbance Climate Change Initiative (CCI...
    downloaded_from:  https://catalogue.ceda.ac.uk/uuid/3628cb2fdba443588155e...
    long_name:        Burned Areas from Fire Climate Change Initiative (FCCI)
    missing_years:    2021
    provider:         ESA CCI
    units:            hectares (ha)

In [29]:
ds_clipped.gwis_ba

<xarray.DataArray 'gwis_ba' (time: 966, latitude: 40, longitude: 40)>
[1545600 values with dtype=float32]
Coordinates:
  * latitude     (latitude) float64 41.88 41.62 41.38 ... 32.62 32.38 32.12
  * longitude    (longitude) float64 -123.9 -123.6 -123.4 ... -114.4 -114.1
    spatial_ref  int64 0
  * time         (time) datetime64[ns] 2001-01-01 2001-01-09 ... 2021-12-27
Attributes:
    aggregation:       Spatio-Temporal | sum
    creator_notes:     Masked ocean with lsm variable. Missing years filled w...
    description:       Global dataset of individual fire perimeters for 2001-...
    downloaded_from::  https://gwis.jrc.ec.europa.eu/apps/country.profile/dow...
    long_name:         Burned Areas from GWIS
    missing_years:     2021
    provider:          Global Wildfire Information System (GWIS)
    units:             hectares (ha)

In [30]:
variables = [
    'tp',
    'rel_hum',
    'ws10',
    't2m_mean',
    't2m_min',
    't2m_max',
    'swvl1',
    'swvl2',
    'swvl3',
    'swvl4',
    'lsm',
    'drought_code_max',
    'drought_code_mean',
    'fwi_max',
    'fwi_mean',
    'lst_day',
    'lai',
    'ndvi',
    'pop_dens',
    'lccs_class_0',
    'lccs_class_1',
    'lccs_class_2',
    'lccs_class_3',
    'lccs_class_4',
    'lccs_class_5',
    'lccs_class_6',
    'lccs_class_7',
    'lccs_class_8',
    'gwis_ba',
    'gwis_ba_valid_mask']
final_ds = ds_clipped[variables]
final_ds = final_ds.where(final_ds.gwis_ba_valid_mask == 1, drop=True)

final_ds.sizes

Frozen({'time': 918, 'latitude': 40, 'longitude': 40})

In [31]:
burned_areas = final_ds['gwis_ba'].values
burned_areas[~np.isnan(burned_areas) & (burned_areas != 0)] = 1
final_ds['gwis_ba'] = xr.DataArray(burned_areas, coords=final_ds['gwis_ba'].coords, dims=final_ds['gwis_ba'].dims)

In [32]:
#final_ds.to_zarr(store = r'/home/aaron/Documents/Wildfire Project/california_dataset.zarr')

In [33]:
final_ds.t2m_mean

<xarray.DataArray 't2m_mean' (time: 918, latitude: 40, longitude: 40)>
array([[[279.8034 , 277.426  , 276.07104, ..., 266.25656, 265.58618,
         265.40552],
        [278.7253 , 277.3053 , 275.957  , ..., 267.26178, 266.7303 ,
         266.65802],
        [278.28677, 276.95218, 275.80075, ..., 267.96402, 267.65088,
         267.99332],
        ...,
        [287.65497, 287.6097 , 287.52814, ..., 289.51898, 289.60468,
         289.55447],
        [287.81644, 287.77402, 287.69608, ..., 289.54495, 289.54303,
         289.54187],
        [287.95132, 287.91776, 287.85327, ..., 289.7045 , 289.5035 ,
         289.4771 ]],

       [[276.83475, 273.80704, 271.61716, ..., 267.47113, 267.68576,
         267.53165],
        [276.04   , 274.05557, 271.31393, ..., 267.23962, 267.73932,
         267.75342],
        [276.3395 , 274.37018, 271.31134, ..., 268.00745, 268.39734,
         268.9311 ],
...
        [288.61987, 288.55023, 288.43713, ..., 288.2193 , 288.41513,
         288.5306 ],
        [288.62863, 288.55304, 288.46027, ..., 288.21994, 288.19293,
         288.28754],
        [288.65005, 288.56992, 288.47165, ..., 288.05   , 287.90662,
         288.1158 ]],

       [[279.35574, 276.84103, 275.67758, ..., 269.03952, 269.49783,
         268.66763],
        [279.79666, 277.16095, 275.62842, ..., 269.26498, 269.967  ,
         269.5994 ],
        [280.1788 , 277.55246, 275.7847 , ..., 269.9172 , 270.09464,
         270.51083],
        ...,
        [288.00992, 287.97913, 287.88654, ..., 286.39532, 286.3222 ,
         286.29645],
        [288.05835, 288.00125, 287.887  , ..., 286.5363 , 286.35782,
         286.19916],
        [288.09436, 288.03323, 287.9247 , ..., 286.48953, 286.27554,
         286.1552 ]]], dtype=float32)
Coordinates:
  * latitude     (latitude) float64 41.88 41.62 41.38 ... 32.62 32.38 32.12
  * longitude    (longitude) float64 -123.9 -123.6 -123.4 ... -114.4 -114.1
    spatial_ref  int64 0
  * time         (time) datetime64[ns] 2001-01-01 2001-01-09 ... 2020-12-10
Attributes:
    aggregation:      Temporal | mean
    description:      This parameter is the temperature of air at 2m above th...
    downloaded_from:  https://cds.climate.copernicus.eu/cdsapp#!/dataset/rean...
    long_name:        2 meters temperature - Mean value
    provider:         ERA5
    units:            Kelvin (K)